In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path_to_augmented_dataset = "F:\\2022-2023\\SEM 1\\CV\\Dataset_1\\Augmented\\"
temp = ['Bowlegs', 'Knock-knee', 'Normal Knee']

In [ ]:
final = pd.DataFrame()
c = 0
for i in temp:
    path_to_folder = path_to_augmented_dataset+i
    for fname in os.listdir(path_to_folder):
        path_to_file = path_to_folder+"\\"+fname
        img = cv2.imread(path_to_file)
        cell_size = (32, 32)  # h x w in pixels
        block_size = (2, 2)  # h x w in cells
        nbins = 9  # number of orientation bins
        hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                          img.shape[0] // cell_size[0] * cell_size[0]),
                                _blockSize=(block_size[1] * cell_size[1],
                                            block_size[0] * cell_size[0]),
                                _blockStride=(cell_size[1], cell_size[0]),
                                _cellSize=(cell_size[1], cell_size[0]),
                                _nbins=nbins)
        descriptor = hog.compute(img)

        out1 = pd.DataFrame(descriptor)
        csv_data = out1.to_csv(r'HOG\HOG_{}.csv'.format(i), mode='a', index=False)

In [ ]:
def read_data(path):
    data = pd.read_csv(path)
    # print(data)
    return data.astype(np.float64)

In [ ]:
p0 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_Bowlegs.csv'
p1 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_Knock-knee.csv'
p2 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_Normal Knee.csv'
pf = 'HOG\HOG_FINAL.csv'

data1 = data_f = read_data(p0)
print(data1.shape)
data2 = read_data(p1)
print(data2.shape)
data3 = read_data(p2)
print(data3.shape)

In [ ]:
inertias = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data1)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,11), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
#performing kmeans on each class

kmeans1 = KMeans(n_clusters=6)
kmeans1.fit(data1)

kmeans2 = KMeans(n_clusters=6)
kmeans2.fit(data2)

kmeans3 = KMeans(n_clusters=6)
kmeans3.fit(data3)


In [ ]:
import pickle

with open("HOG_model1.pkl", "wb") as f:
    pickle.dump(kmeans1, f)
with open("HOG_model2.pkl", "wb") as f:
    pickle.dump(kmeans2, f)
with open("HOG_model3.pkl", "wb") as f:
    pickle.dump(kmeans3, f)

In [ ]:
final = pd.DataFrame()
c = 0
for i in temp:
    data = []
    path_to_folder = path_to_augmented_dataset+i
    for fname in os.listdir(path_to_folder):
        path_to_file = path_to_folder+"\\"+fname
        img = cv2.imread(path_to_file)
        cell_size = (32, 32)  # h x w in pixels
        block_size = (2, 2)  # h x w in cells
        nbins = 9  # number of orientation bins
        hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                          img.shape[0] // cell_size[0] * cell_size[0]),
                                _blockSize=(block_size[1] * cell_size[1],
                                            block_size[0] * cell_size[0]),
                                _blockStride=(cell_size[1], cell_size[0]),
                                _cellSize=(cell_size[1], cell_size[0]),
                                _nbins=nbins)
        descriptor = hog.compute(img)
        out1 = pd.DataFrame(descriptor)
        array_double = np.array(out1, dtype=np.double)
        if i == temp[0]:
            a=kmeans1.predict(array_double)
        elif i == temp[1]:
            a = kmeans2.predict(array_double)
        else:
            a = kmeans3.predict(array_double)
        hist=np.histogram(a,bins=6)
        print("HIST DATA", hist)

        data.append(hist[0])
    Output = pd.DataFrame(data)
    Output["Class"] = c
    csv_data=Output.to_csv(r'HOG\HOG_FINAL_{}.csv'.format(i), mode='a', index=False)
    c += 1


In [ ]:
pf0 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_FINAL_Bowlegs.csv'
pf1 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_FINAL_Knock-knee.csv'
pf2 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_FINAL_Normal Knee.csv'
pfd = 'HOG\HOG_FINAL.csv'
d1 = fd = read_data(pf0)
print(d1)
d2 = read_data(pf1)
print(d2)
d3 = read_data(pf2)
print(d3)
fd = fd.append(d2)
fd = fd.append(d3)
print(fd)
csv_data = fd.to_csv(pfd, mode='a', index=False)

In [ ]:
data = read_data(pfd)
X = data.iloc[:, :-1]
print(X)
Y = data.iloc[:, -1]
print(Y)

# train test split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, train_size=0.8)
print(train_X)
print(test_X)

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 7))
# ax.spines['top'].set_visible(False)
# ax.spines['left'].set_visible(False)
# ax.spines['right'].set_visible(False)
# # adding major gridlines
# ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
# ax.scatter(train_X, train_Y, color="#8C7298")
# plt.show()

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score

sc = StandardScaler()
train_X_KNN = sc.fit_transform(train_X)
test_X_KNN = sc.transform(test_X)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

In [ ]:
classifier.fit(train_X_KNN, train_Y)

In [ ]:
y_pred = classifier.predict(test_X_KNN)

cm = confusion_matrix(test_Y, y_pred)
ac = accuracy_score(test_Y,y_pred)
print(cm)
print(ac)

# XgBoost

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
model = XGBClassifier()
sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)
model.fit(train_X, train_Y)
print(model)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(test_X)
# evaluate predictions
accuracy = accuracy_score(test_Y, y_pred)
print(accuracy)

# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

classifier_SVM = SVC(kernel='linear')
# classifier_SVM = SVC(kernel='sigmoid')
# classifier_SVM = SVC(kernel='linear')

In [ ]:
classifier_SVM.fit(train_X, train_Y)

In [ ]:
predictions = classifier_SVM.predict(test_X)
cm = confusion_matrix(test_Y, predictions)
ac = accuracy_score(test_Y,y_pred)

In [ ]:
print(cm, ac)